<a href="https://colab.research.google.com/github/gandhirajj/Symptom-based-food-Advisor/blob/main/Symptom_Based_Food_Advisor_for_Better_Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

In [ ]:
# Load dataset
data = pd.read_csv("/content/disease_food2_2.csv")
data.head()


,disease,foods,nutrients
0,Fever,"Fever – Warm soups, boiled rice, yogurt, cocon...","Fever – Warm soups keep the body hydrated, boi..."
1,Cold,"Cold – Chicken soup, citrus fruits, honey, gin...","Cold – Chicken soup clears nasal passages, cit..."
2,Cough,"Cough – Turmeric milk, honey, ginger, almonds,...",Cough – Turmeric milk has antibacterial proper...
3,Flu,"Flu – Bone broth, garlic, green tea, berries, ...","Flu – Bone broth supports immunity, garlic has..."
4,Sore throat,"Sore throat – Warm saltwater gargle, chamomile...",Sore Throat – Warm saltwater gargle kills bact...


In [ ]:
data.tail()

,disease,foods,nutrients
698,Wound Infection,Wound Infection – Healing-supporting foods: pr...,Wound Infection – Healing-supporting foods: pr...
699,X-Linked Agammaglobulinemia,X-Linked Agammaglobulinemia – Immune-supportin...,X-Linked Agammaglobulinemia – Immune-supportin...
700,Xeroderma Pigmentosum,Xeroderma Pigmentosum – Skin-protecting foods:...,Xeroderma Pigmentosum – Skin-protecting foods:...
701,Yellow Fever,Yellow Fever – Hydrating and immune-boosting f...,Yellow Fever – Hydrating and immune-boosting f...
702,Zika Virus Infection,Zika Virus Infection – Immune-boosting and hyd...,Zika Virus Infection – Immune-boosting and hyd...


In [ ]:
# Tokenization
num_words = 5000
tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
tokenizer.fit_on_texts(data['disease'])

sequences = tokenizer.texts_to_sequences(data['disease'])


In [ ]:
print(data.columns)


Index(['disease', 'foods', 'nutrients'], dtype='object')


In [ ]:
max_length = max(len(x) for x in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')


In [ ]:
# Encoding the labels
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

disease_labels = label_encoder_disease.fit_transform(data['foods'])
prescription_labels = label_encoder_prescription.fit_transform(data['nutrients'])

# Converting labels to categorical
disease_labels_categorical = to_categorical(disease_labels)
prescription_labels_categorical = to_categorical(prescription_labels)


In [ ]:
Y = np.hstack((disease_labels_categorical, prescription_labels_categorical))


In [ ]:
input_layer = Input(shape=(max_length,))

embedding = Embedding(input_dim=5000, output_dim=64)(input_layer)
lstm_layer = LSTM(64)(embedding)

disease_output = Dense(len(label_encoder_disease.classes_), activation='softmax',
name='disease_output')(lstm_layer)

prescription_output = Dense(len(label_encoder_prescription.classes_),
activation='softmax', name='prescription_output')(lstm_layer)


In [ ]:
model = Model(inputs=input_layer, outputs=[disease_output, prescription_output])

model.compile(
    loss={'disease_output': 'categorical_crossentropy',
    'prescription_output': 'categorical_crossentropy'},
    optimizer='adam',
    metrics={'disease_output': ['accuracy'], 'prescription_output': ['accuracy']}
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 5, 64)     │    320,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     33,024 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ disease_output      │ (None, 703)       │     45,695 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prescription_output │ (None, 703)       │     45,695 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 444,414 (1.70 MB)

 Trainable params: 444,414 (1.70 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(padded_sequences, {'disease_output': disease_labels_categorical, 'prescription_output':
      prescription_labels_categorical}, epochs=100, batch_size=32)


Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - disease_output_accuracy: 0.0000e+00 - disease_output_loss: 6.5580 - loss: 13.1157 - prescription_output_accuracy: 0.0000e+00 - prescription_output_loss: 6.5577
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - disease_output_accuracy: 0.0022 - disease_output_loss: 6.5554 - loss: 13.1109 - prescription_output_accuracy: 0.0124 - prescription_output_loss: 6.5554
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - disease_output_accuracy: 0.0058 - disease_output_loss: 6.5536 - loss: 13.1071 - prescription_output_accuracy: 0.0090 - prescription_output_loss: 6.5535
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - disease_output_accuracy: 0.0137 - disease_output_loss: 6.5507 - loss: 13.1014 - prescription_output_accuracy: 0.0121 - prescription_output_loss: 6.5507
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - disease_output_accuracy: 0.0670 - disease_output_loss: 6.5418 - loss: 13.0841 - prescription_output_accuracy: 0.03

In [ ]:
def make_prediction(patient_problem):
    # Preprocessing the input
    sequence = tokenizer.texts_to_sequences([patient_problem])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

    # Making prediction
    prediction = model.predict(padded_sequence)

    # Decoding the prediction
    disease_index = np.argmax(prediction[0], axis=1)[0]
    prescription_index = np.argmax(prediction[1], axis=1)[0]

    disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
    prescription_predicted = label_encoder_prescription.inverse_transform([prescription_index])[0]

    print(f"Suggested Foods: {disease_predicted}")
    print(f"Nutients in that food: {prescription_predicted}")


patient_input = "Chemical Pneumonitis"
make_prediction(patient_input)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step
Suggested Foods: Chemical Pneumonitis – Green tea, ginger, turmeric, leafy greens, citrus fruits
Nutients in that food: Chemical Pneumonitis – Green tea, ginger, and turmeric fight lung inflammation, leafy greens assist in detoxification, and citrus fruits enhance immunity.


In [ ]:
# Save model
model.save("disease_food_model.h5")

# Save tokenizer, label encoders, and max_length
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

with open("label_encoders.pkl", "wb") as f:
    pickle.dump((label_encoder_disease, label_encoder_prescription), f)

with open("maxlen.pkl", "wb") as f:
    pickle.dump(max_length, f)
